## Installation

In [1]:
!pip install semantic-version
!pip install huggingface_hub
!pip install transformers peft
!pip install vllm

INFO: pip is looking at multiple versions of model-hosting-container-standards to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.9/87.9 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.5/466.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.0/355.0 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.0/183.0 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 111.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 125.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 70.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Connecting To Hugging Face and Drive

In [ ]:
from huggingface_hub import login

login(token="")

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

## Loading Model From Hugging Face with VLLM

In [ ]:
import json
import time
import re
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer

MODEL_ID = "mahdishahsavari/gpt-oss-20B-finetune-merged-mxfp4"

print("⏳ در حال لود کردن مدل روی GPU...")

# پارامترهای لود مدل (تنظیمات سخت‌افزاری)
llm = LLM(
    model=MODEL_ID,
    # GPT-OSS 20B احتمالا نیاز به فضای کمتری دارد. اگر با float16 مشکل داشتید، از 'auto' استفاده کنید
    dtype="bfloat16",
    # اگر ارور حافظه دادید، این را تا 4096 کاهش دهید
    max_model_len=4096,
    # در کولب تک کارته حتما باید ۱ باشد
    tensor_parallel_size=1,
    gpu_memory_utilization=0.95,
    trust_remote_code=True,
    enforce_eager=True # برای کنترل بهتر OOM
)

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_ID,
    trust_remote_code=True
)

print("✅ مدل با موفقیت لود شد.")

⏳ در حال لود کردن مدل روی GPU...
INFO 12-06 10:33:23 [utils.py:253] non-default args: {'trust_remote_code': True, 'dtype': 'bfloat16', 'seed': None, 'max_model_len': 4096, 'gpu_memory_utilization': 0.95, 'disable_log_stats': True, 'enforce_eager': True, 'model': 'mahdishahsavari/gpt-oss-20B-finetune-merged-mxfp4'}


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


config.json: 0.00B [00:00, ?B/s]

WARNING 12-06 10:33:24 [arg_utils.py:1175] `seed=None` is equivalent to `seed=0` in V1 Engine. You will no longer be allowed to pass `None` in v0.13.
INFO 12-06 10:33:43 [model.py:637] Resolved architecture: GptOssForCausalLM


model.safetensors.index.json: 0.00B [00:00, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/27.9M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

WARNING 12-06 10:33:50 [system_utils.py:136] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/usage/troubleshooting.html#python-multiprocessing for more information. Reasons: CUDA is initialized
INFO 12-06 10:35:21 [llm.py:343] Supported tasks: ['generate']
✅ مدل با موفقیت لود شد.


## Prompt Engineering

In [ ]:
SYSTEM_PROMPT = """ """

input_data = {
    "input_block": {
        "customer_profile": {
            ""
        },
        "underwriting": {
            ""
        },
        "churn": {
            ""
        },
        "constraints": {
            ""
        }
    }
}



input_json_str = json.dumps(input_data, ensure_ascii=False)

# ساختار پیام: (Chat Format)
messages = [
    {
        "role": "developer",
        "content": SYSTEM_PROMPT,
    },
    {
        "role": "user",
        "content": "--- INPUT DATA ---\n" + input_json_str,
    },
]

# اعمال Chat Template
final_prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

## Inference

In [33]:
# پارامترهای نمونه‌برداری (Sampling Parameters)
sampling_params = SamplingParams(
    temperature=0.0,              # صفر برای منطق مطلق و خروجی JSON
    top_p=1.0,
    repetition_penalty=1.0,       # جلوگیری از تکرار (طبق درخواست شما)
    max_tokens=4096,              # فضای کافی برای جیسون خروجی
    # ignore_eos=True
    # stop=["<|im_end|>", "<|endoftext|>"]
)

## Production Run

In [34]:
print("\n🚀 در حال تولید پاسخ...")
start_time = time.time()

outputs = llm.generate([final_prompt], sampling_params)

end_time = time.time()

generated = outputs[0].outputs[0]
generated = outputs[0].outputs[0]
print("Finish reason:", generated.finish_reason)
print("Num tokens:", len(generated.token_ids))
generated_text = generated.text
token_count = len(generated.token_ids)
duration = end_time - start_time

print("="*60)
print(f"⏱️ زمان سپری شده: {duration:.2f} ثانیه")
print(f"⚡ سرعت (Throughput): {token_count / duration:.2f} توکن/ثانیه")
print("="*60)

print("\n--- Raw Output ---")
print(generated_text)

print("\n--- JSON Validation Check ---")
try:
    raw = generated_text

    # 1) پیدا کردن اولین {
    start = raw.find("{")
    if start == -1:
        raise ValueError("هیچ آکولاد باز { در خروجی پیدا نشد (مدل JSON را شروع نکرد).")

    # 2) پیدا کردن آخرین }
    end = raw.rfind("}")
    if end == -1:
        # یعنی JSON را شروع کرده ولی کامل نکرده
        print("❌ مدل JSON را نیمه‌کاره رها کرده (هیچ آکولاد بسته } پیدا نشد).")
        print("بخش شروع JSON که تولید شده:")
        print(raw[start:])
        raise ValueError("JSON ناقص است و قابل parse کردن نیست.")

    json_str = raw[start:end+1]

    # 3) فیکس ویرگول فارسی به‌عنوان delimiter بین فیلدها
    #    "..."، "..."  →  "..." , "..."
    json_str = re.sub(r"\"،\s+\"", "\", \"", json_str)

    # 4) تلاش برای parse
    parsed_json = json.loads(json_str)

    print("✅ خروجی یک JSON معتبر است! در زیر فرمت زیبا شده را ببینید:\n")
    print(json.dumps(parsed_json, indent=4, ensure_ascii=False))

except Exception as e:
    print("❌ خطا: خروجی JSON معتبر نیست.")
    print("خام خروجی مدل:")
    print(raw)
    print(f"\nجزئیات خطا: {e}")



🚀 در حال تولید پاسخ...


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Finish reason: stop
Num tokens: 484
⏱️ زمان سپری شده: 15.74 ثانیه
⚡ سرعت (Throughput): 30.75 توکن/ثانیه

--- Raw Output ---
{"summary": "ریسک کلی اداری با پوشش آتش‌سوزی و زلزله در سطح Low است، اما نرخ بالای churn (82٪) و نیاز به «خدمات سریع» و «تغییرات فنی» برای نگهداشت مشتری است. پیشنهاد: تمرکز بر سرعت خدمت‌رسانی و بازنگری پوشش‌های اضافی.", "retention_plan": [{"step": 1, "action": "ارتقاء SLA پاسخ‌گویی به درخواست‌های نگهداری و تعمیرات (مثلاً ۲ ساعت در روز کاری)", "channel": "تماس تلفنی/ایمیل", "timing": "در همان روز درخواست", "notes": "به‌منظور کاهش ریزش مشتری به دلیل تأخیر در خدمات."}, {"step": 2, "action": "ارائه گزارش مهندسی ریسک برای تأیید سازه فلزی و اقدامات حفاظتی (مانند پوشش ضدآتش‌سوزی)", "channel": "جلسه آنلاین/حضوری", "timing": "در ۱ هفته از دریافت درخواست", "notes": "برای اطمینان از ایمنی سازه در برابر آتش و زلزله."}, {"step": 3, "action": "ارائه تخفیف ۵٪ در پوشش زلزله به‌عنوان انگیزه نگهداشت", "channel": "ایمیل/پلیت‌کارت", "timing": "در همان زمان ارائه پیشنهاد", "notes": "ب

In [ ]:
prompt_token_ids = tokenizer.encode(final_prompt)
print("Prompt token length:", len(prompt_token_ids))

Prompt token length: 1635
